# Analisis de performance para cada paciente

In [1]:
import sys
import os

import torchaudio.transforms as T
import pandas                as pd
import numpy                 as np

from torchvision       import transforms

sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, smoothing_label
from Model             import iESPnet
from TrainEval         import test_model_dsf_iespnet_abl_perpatient, get_performance_indices

sys.path.append(os.path.abspath(os.path.join('../../..','03 Dynamic-Spatial-Filtering')))
from models            import DynamicSpatialFilter

In [2]:
# direccion donde se encuentran los espectrogramas 
SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)

In [3]:
# Variables iESPnet
FREQ_MASK_PARAM       = 10
TIME_MASK_PARAN       = 20
N_CLASSES             = 1
learning_rate_iespnet = 1e-3
batch_size            = 64    #128
epochs                = 20
num_workers           = 4

In [4]:
# Variables DSF
denoising             = 'autoreject'   # 'autoreject' 'data_augm' 
model                 = 'stager_net'
dsf_type              = 'dsfd'         # 'dsfd' 'dsfm_st'
mlp_input             = 'log_diag_cov'
dsf_soft_thresh       = False
dsf_n_out_channels    = None
n_channels            = 4
learning_rate_dsf     = 1e-2

In [5]:
# hiperparametros iESPnet y DSF
hparams = {
           "n_cnn_layers"          : 3,
           "n_rnn_layers"          : 3,
           "rnn_dim"               : [150, 100, 50],
           "n_class"               : N_CLASSES,
           "out_ch"                : [8,8,16],
           "dropout"               : 0.3,
           "learning_rate_iespnet" : learning_rate_iespnet,
           "learning_rate_dsf"     : learning_rate_dsf,
           "batch_size"            : batch_size,
           "num_workers"           : num_workers,
           "epochs"                : epochs
          }

In [6]:
model1 = DynamicSpatialFilter(
                              n_channels, 
                              mlp_input            = mlp_input, 
                              n_out_channels       = dsf_n_out_channels, 
                              apply_soft_thresh    = dsf_soft_thresh
                             )

In [7]:
model2 = iESPnet(
                 hparams['n_cnn_layers'],
                 hparams['n_rnn_layers'],
                 hparams['rnn_dim'],
                 hparams['n_class'],
                 hparams['out_ch'],
                 hparams['dropout'],
                )

In [8]:
save_path        = '/media/martin/Disco2/Rns_Data/experimentos/dsf_iespnet_noprocess/exp3/'
experiment       = 'exp3.2'
save_models      = save_path + experiment + '/models/'
save_predictions = save_path + experiment + '/results/'
outputfile       = save_models + 'model_' + experiment
best_path        = outputfile + '.pth'

In [9]:
test_id  = ['PIT-RNS1090', 'PIT-RNS8973', 'PIT-RNS1438', 'PIT-RNS8326', 'PIT-RNS3016']
vali_id  = ['PIT-RNS1603', 'PIT-RNS1556', 'PIT-RNS1534', 'PIT-RNS6989', 'PIT-RNS2543', 'PIT-RNS7168', 'PIT-RNS6762']


train_df = df_meta.copy()
test_df  = pd.DataFrame()
vali_df  = pd.DataFrame()

for s in range (len(test_id)):
    test_df = pd.concat([test_df, df_meta[df_meta['rns_id'] == test_id[s]]])
    test_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == test_id[s]].index, inplace = True)

for s in range(len(vali_id)):
    vali_df=pd.concat([vali_df, df_meta[df_meta['rns_id'] == vali_id[s]]])
    vali_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == vali_id[s]].index, inplace = True)

In [14]:
patients_train = train_df['rns_id'].unique().tolist()
patients_test  = test_df['rns_id'].unique().tolist()
patients_vali  = vali_df['rns_id'].unique().tolist()

In [11]:
best_thr = 0.2

In [14]:
print('in training: ', patients_train[s] )

in training:  PIT-RNS9183


In [ ]:
for s in range (len(patients_train)):
    # Dataloaders creados
    train_data = SeizureDatasetLabelTimev2(
                                           file             = train_df[train_df['rns_id'] == patients_train[s]],
                                           root_dir         = SPE_DIR,
                                           transform        = None, 
                                           target_transform = smoothing_label(),
                                          )
    
    print()
    print('in training')
    # in training
    outputs_train = test_model_dsf_iespnet_abl_perpatient(model1, model2, hparams, best_path, train_data)
    prediction_tr = get_performance_indices(outputs_train['y_true'], outputs_train['y_prob'], best_thr)

    predict_ = { 
                "prediction_tr"   : prediction_tr,
               }
    
    np.save(save_predictions + patients_train[s] + '_results.npy', predict_)

In [13]:
vali_df[vali_df['rns_id'] == patients_vali[s]]

,rns_id,data,label,time


In [16]:
for s in range (len(patients_vali)):
    # testing data should be balanced, just be "as it is"
    vali_data  = SeizureDatasetLabelTimev2(
                                            file             = vali_df[vali_df['rns_id'] == patients_vali[s]],
                                            root_dir         = SPE_DIR,
                                            transform        = None,
                                            target_transform = smoothing_label()  
                                            )

    print()
    print('in validation: ',patients_vali[s])
    # in validation
    outputs_vali = test_model_dsf_iespnet_abl_perpatient(model1, model2, hparams, best_path, vali_data)
    prediction_va = get_performance_indices(outputs_vali['y_true'], outputs_vali['y_prob'], best_thr)

    predict_ = { 
                "prediction_va"   : prediction_va,
                }

    np.save(save_predictions + patients_vali[s] + '_results.npy', predict_)

    del vali_data


in validation:  PIT-RNS1603
Using cuda device
torch.Size([64, 4, 22500])
torch.Size([64, 4, 22500])
torch.Size([64, 4, 22500])


KeyboardInterrupt: 

In [ ]:
for s in range (len(patients_test)):
    # testing data should be balanced, just be "as it is"
    test_data  = SeizureDatasetLabelTimev2(
                                           file             = test_df,
                                           root_dir         = SPE_DIR,
                                           transform        = None,
                                           target_transform = smoothing_label()  
                                          )
    
    print()
    print('in testing')
    # in testing
    outputs_test  = test_model_dsf_iespnet_abl_perpatient(model1, model2, hparams, best_path, test_data)
    prediction_te = get_performance_indices(outputs_test['y_true'], outputs_test['y_prob'], best_thr)

    predict_ = { 
                "prediction_te"   : prediction_te,
               }
    
    np.save(save_predictions + patients_test[s] + '_results.npy', predict_)
        


In [ ]:
# validation data should be balanced, just be "as it is"
tes_data  = SeizureDatasetLabelTimev2(
                                       file             = vali_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None,
                                       target_transform = smoothing_label()  
                                      )